<a href="https://colab.research.google.com/github/Futaba-Kosuke/STL10/blob/develop/testing/main_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import torch
import torchvision
from torchvision import transforms

In [0]:
# ROOT = './'
ROOT = './drive/My Drive/STL10/'

In [25]:
# GPUの確認
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [0]:
batch_size = 10

In [27]:
# データの読み込み
transform = transforms.Compose(
  [transforms.Resize((300, 300)),
   transforms.ToTensor(),
   transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]
)
data_set = torchvision.datasets.ImageFolder(
  root=ROOT + 'images',
  # root='./images',
  transform=transform
)
print('data_size: ', len(data_set))

data_loader = torch.utils.data.DataLoader(
  data_set,
  batch_size=batch_size,
  shuffle=False,
  num_workers=1
)
print('Finish load images')

classes = ('airplane', 'bird', 'car', 'cat', 'deer', 'dog', 'horse', 'monkey', 'ship', 'truck')

data_size:  1000
Finish load images


In [0]:
# モデルの読み込み
# アンサンブル学習を行う為、複数のモデルを読み込む
from torch import nn
from torchvision import models

vgg16 = models.vgg16()
densenet = models.densenet161()
wide_resnet = models.wide_resnet50_2()

In [0]:
from torch import nn
import torch.nn.functional as F

class EnsembleNet(nn.Module):
    def __init__(self, net_1=densenet, net_2=vgg16, net_3=wide_resnet, nb_classes=10):
        super(EnsembleNet, self).__init__()
        self.net_1 = net_1
        self.net_2 = net_2
        self.net_3 = net_3

        # Remove last linear layer
        last_in_features_sum = self.net_1.classifier.in_features + self.net_2.classifier[6].in_features + self.net_3.fc.in_features
        self.net_1.classifier = nn.Identity()
        self.net_2.classifier[6] = nn.Identity()
        self.net_3.fc = nn.Identity()
        
        # Create new classifier
        self.classifier = nn.Linear(last_in_features_sum, nb_classes)
        
    def forward(self, x):
        x_1 = self.net_1(x.clone())  # clone to make sure x is not changed by inplace methods
        x_1 = x_1.view(x_1.size(0), -1)

        x_2 = self.net_2(x.clone())
        x_2 = x_2.view(x_2.size(0), -1)
        
        x_3 = self.net_3(x)
        x_3 = x_3.view(x_3.size(0), -1)
        
        x = torch.cat((x_1, x_2, x_3), dim=1)
        
        x = self.classifier(F.relu(x))
        return x

In [30]:
net = EnsembleNet()
net.load_state_dict(torch.load(ROOT + 'models/ensemble.pth', map_location=device))
net = net.to(device)

print('Finish load models')

Finish load models


In [32]:
# 検知開始
net.eval()

class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))

with torch.no_grad():
  for (inputs, labels) in data_loader:
    inputs = inputs.to(device)
    labels = labels.to(device)

    outputs = net(inputs)

    _, predicted = torch.max(outputs, 1)
    is_correct = (predicted == labels).squeeze()

    for idx, label in enumerate(labels):
      class_correct[label] += is_correct[idx].item()
      class_total[label] += 1

# 正解率の出力
for i in range(10):
  print('Accuracy of %5s : %02.01f %%' % (
    classes[i], 100 * class_correct[i] / class_total[i]))
  
print('\nTop-1 Accuracy: %02.01f %%' % (100 * sum(class_correct) / sum(class_total)))

Accuracy of airplane : 100.0 %
Accuracy of  bird : 100.0 %
Accuracy of   car : 100.0 %
Accuracy of   cat : 100.0 %
Accuracy of  deer : 100.0 %
Accuracy of   dog : 100.0 %
Accuracy of horse : 100.0 %
Accuracy of monkey : 100.0 %
Accuracy of  ship : 100.0 %
Accuracy of truck : 100.0 %

Top-1 Accuracy: 100.0 %
